In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re, os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from collections import OrderedDict
import pickle

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keybert import KeyBERT

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import matplotlib.pyplot as plt
import spacy
import string
from sklearn.metrics.pairwise import cosine_similarity

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
#list(gensim.downloader.info()['models'].keys())
#wv = api.load('glove-wiki-gigaword-300')
#wv.save('glove_wiki_vectors.kv')

In [5]:
# Define the relative paths
relative_path_vec = "./glove_wiki_vectors.kv"
relative_path_embedding = "./oov_word_embeddings.pkl"
relative_path_sdg = "./sdg_key.csv"
relative_path_input = "../sdg_data.csv"

# Get the directory of the current Python file being executed if available, else use the current working directory
current_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()

# Get the absolute paths by joining the current directory with the relative paths
absolute_path_vec = os.path.normpath(os.path.join(current_dir, relative_path_vec))
absolute_path_embedding = os.path.normpath(os.path.join(current_dir, relative_path_embedding))
absolute_path_sdg = os.path.normpath(os.path.join(current_dir, relative_path_sdg))
absolute_path_input = os.path.normpath(os.path.join(current_dir, relative_path_input))

In [6]:
# Load the saved word embeddings dictionary from the file
with open(absolute_path_embedding, 'rb') as f:
    oov_word_embeddings = pickle.load(f)

In [7]:
wv = KeyedVectors.load(absolute_path_vec)

In [8]:
kw_model = KeyBERT(model=wv)

In [9]:
IGNORE_WORDS = ['including', 'proportion', 'whose', 'number', 'per', 'no', 'under', 'use', 'type', 'percentage', 'total', 'data', 'year', 'due', 'rate', 'hects', 'hect', 'noof']

In [10]:
# Creating our tokenizer function
def cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()
    ignore_word = []
    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        #letters_list = list(set(letters))
        letters_list = list(OrderedDict.fromkeys(letters))   
        cleaned.append(letters_list)

    return cleaned

In [11]:
# Creating our tokenizer function
def concat_cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()

    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        #letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        #IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        #letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        letters = ' '.join(letters)
        cleaned.append(letters)

    return cleaned

In [12]:
def process_keyword(data):
    result = []
    for sublist in data:
        inner_list = []
        for word, _ in sublist:
            # Split the word if it contains a space and reverse the order of words
            words = word.split()
            # Filter out numeric values
            words = [w for w in words if not w.isdigit()]
            if words:  # Exclude empty lists
                inner_list.append(words)
        inner_list.reverse()
        result.append(inner_list)
    return result

In [13]:
def add_elements(row):
    if not row['keywords']:  # Check if col2 is empty
        row['keywords'].extend([[elem] for elem in row['new_description']])  # Add elements from col1 to col2 as lists
    return row

In [14]:
sdg = pd.read_csv(absolute_path_input, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [15]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [16]:
sdg.columns

Index(['Goal No.', 'Goal', 'Nodal Department', 'Targets',
       'Other Related Major Departments', 'Tentative Indicators', 'Target_id',
       'Indicator_id'],
      dtype='object')

In [17]:
sdg.drop(['Nodal Department','Other Related Major Departments'],axis=1,inplace=True)

In [18]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [19]:
sdg.reset_index(inplace = True, drop = True)

In [20]:
sdg

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1


In [21]:
goals = list(zip(sdg['Goal No.'], sdg['Goal']))
goals_set = list(OrderedDict.fromkeys(goals))
goals_set

[('1', 'End poverty in all its forms everywhere'),
 ('2',
  'End hunger, achieve food security and improved nutrition and promote sustainable agriculture'),
 ('3', 'Ensure healthy lives and promote well-being for all at all ages'),
 ('4',
  'Ensure inclusive and equitable quality education and promote lifelong learning opportunities\nfor all'),
 ('5', 'Achieve gender equality and empower all women and\ngirls'),
 ('6',
  'Ensure availability and sustainable management of water and sanitation for all'),
 ('7',
  'Ensure access to affordable, reliable, sustainable and modern energy\nfor all'),
 ('8',
  'Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all'),
 ('9',
  'Build resilient infrastructure, promote inclusive and sustainable industrializatio n and foster innovation'),
 ('10', 'Reduce inequality within the State'),
 ('11',
  'Make cities and human settlements inclusive, safe,\nresilient and sustainable'),
 ('12', 'Ensu

In [22]:
targets = list(zip(sdg['Target_id'], sdg['Targets']))
targets_set = list(OrderedDict.fromkeys(targets))
targets_set

[('1.1',
  'By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day.'),
 ('1.2',
  'By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions.'),
 ('1.3',
  'Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable.'),
 ('1.4',
  'By 2030, ensure that all men and women, in particular the poor and the vulnerable, have equal rights to economic resources, as well as access to basic services, ownership and control over land and other forms of property, inheritance, natural resources, appropriate new technology and financial services, including microfinance.'),
 ('1.5',
  'By 2030, build the resilience of the poor and those in vulnerable situations and reduce their exposure and vulnerability to climate-related e

In [23]:
indicators = list(zip(sdg['Indicator_id'], sdg['Tentative Indicators']))
indicators_set = list(OrderedDict.fromkeys(indicators))
indicators_set

[('1.1.1',
  'Proportion of the population below the international poverty line, disaggregated by sex, age group, employment status and geographical location (urban/rural).'),
 ('1.2.1',
  'Proportion of the population living below the national poverty line, disaggregated by sex and age group'),
 ('1.2.2',
  'Proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions.'),
 ('1.3.1',
  'Percentage of the population covered by social protection floors/systems, disaggregated by sex, and distinguishing children, the unemployed, old-age persons, persons with disabilities, pregnant women/ newborns, work injury victims, the poor and the vulnerable.'),
 ('1.4.1',
  'Proportion of the population living in households with access to basic services.'),
 ('1.5.1',
  'Number of deaths, missing people, injured, relocated or evacuated due to disasters per 100,000 people.'),
 ('1.6.1',
  'Number of state action plans related to multilatera

In [24]:
sdg_list = goals_set + targets_set + indicators_set

In [25]:
sdg_new = pd.DataFrame(sdg_list, columns = ['Id', 'Description'])
sdg_new

,Id,Description
0,1,End poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved..."
2,3,Ensure healthy lives and promote well-being fo...
3,4,Ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...
...,...,...
322,16.4.1,Total amount of approved funding to promote th...
323,16.5.1,Proportion of individuals using the Internet.
324,17.1.1,Proportion of sustainable development indicato...
325,17.2.1,Dollar value of all resources made available t...


In [26]:
sdg_new['cleaned_descp'] = sdg_new['Description'].apply(lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')).strip())

In [27]:
sdg_new['cleaned_descp'][11]

'Ensure sustainable consumption and production patterns'

In [28]:
sdg_new = sdg_new.drop(columns={'Description'})
sdg_new

,Id,cleaned_descp
0,1,End poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved..."
2,3,Ensure healthy lives and promote well-being fo...
3,4,Ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...
...,...,...
322,16.4.1,Total amount of approved funding to promote th...
323,16.5.1,Proportion of individuals using the Internet.
324,17.1.1,Proportion of sustainable development indicato...
325,17.2.1,Dollar value of all resources made available t...


In [29]:
sdg_new.rename(columns={'cleaned_descp':'Description'}, inplace=True)
sdg_new

,Id,Description
0,1,End poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved..."
2,3,Ensure healthy lives and promote well-being fo...
3,4,Ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...
...,...,...
322,16.4.1,Total amount of approved funding to promote th...
323,16.5.1,Proportion of individuals using the Internet.
324,17.1.1,Proportion of sustainable development indicato...
325,17.2.1,Dollar value of all resources made available t...


In [30]:
texto_data=sdg_new['Description'].tolist()
process_text=cleaning(texto_data)

cleaned_sent = pd.DataFrame({'new_description': process_text})
sdg_new['new_description']=cleaned_sent

In [31]:
texto_data0=sdg_new['Description'].tolist()
process_text0=concat_cleaning(texto_data0)

cleaned_sent0 = pd.DataFrame({'new_description_str': process_text0})
sdg_new['new_description_str']=cleaned_sent0

In [32]:
type(sdg_new['new_description_str'].loc[0])

str

In [33]:
sdg_new

,Id,Description,new_description,new_description_str
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...
...,...,...,...,...
322,16.4.1,Total amount of approved funding to promote th...,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...
323,16.5.1,Proportion of individuals using the Internet.,"[individual, using, internet]",proportion of individuals using the internet
324,17.1.1,Proportion of sustainable development indicato...,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...
325,17.2.1,Dollar value of all resources made available t...,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...


In [34]:
sdg_sem = sdg_new.copy()

word_count = sdg_sem['new_description'].apply(lambda x: len(x))
word_count

0       4
1      10
2       6
3       9
4       6
       ..
322    11
323     3
324    15
325     8
326     3
Name: new_description, Length: 327, dtype: int64

In [35]:
median_indicator = np.median(word_count)
print(median_indicator)

mean_indicator = np.mean(word_count)
print(mean_indicator)

10.0
10.951070336391437


In [36]:
stopwords_set = list(set(nltk.corpus.stopwords.words('english')))
stopwords_set.extend(IGNORE_WORDS)
stopwords_set = list(set(stopwords_set))
stopwords_set

['but',
 'y',
 'some',
 'data',
 'did',
 "you've",
 "wasn't",
 'where',
 'doesn',
 'a',
 'of',
 'on',
 "shan't",
 'which',
 'should',
 'aren',
 'themselves',
 'during',
 'those',
 'up',
 'i',
 "shouldn't",
 "hadn't",
 'too',
 'through',
 'having',
 'then',
 'when',
 'her',
 'it',
 'no',
 've',
 'own',
 "haven't",
 'haven',
 'won',
 'hect',
 'whom',
 'ourselves',
 'again',
 'as',
 'that',
 "won't",
 'with',
 'needn',
 'doing',
 'yourself',
 'just',
 'me',
 'what',
 'be',
 'was',
 'have',
 'whose',
 "couldn't",
 'mustn',
 'why',
 'by',
 'further',
 'now',
 're',
 'below',
 'or',
 'their',
 'being',
 'such',
 'once',
 'nor',
 'does',
 'am',
 'been',
 "mightn't",
 "it's",
 'these',
 'any',
 'shan',
 'wouldn',
 'noof',
 'mightn',
 'and',
 'against',
 'under',
 'per',
 'there',
 'than',
 "needn't",
 'didn',
 'few',
 'ours',
 'year',
 'you',
 'herself',
 'from',
 "mustn't",
 'number',
 'they',
 'due',
 'theirs',
 'hadn',
 'm',
 "weren't",
 'my',
 'out',
 "don't",
 'him',
 "you'd",
 'over',
 '

In [37]:
#using 'Tentative Indicators', takin top_n=8 as mode = 8 and nr_candidates=10 as mean = 9.2
#sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['Tentative Indicators'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=8)

#using 'new_indicators_str', takin top_n=6 as median = 8 and nr_candidates=10 as mean = 9.2
sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['new_description_str'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=6)

sdg_sem['keywords'] = process_keyword(sdg_sem['keywords'].tolist())
sdg_sem

,Id,Description,new_description,new_description_str,keywords
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge..."
...,...,...,...,...,...
322,16.4.1,Total amount of approved funding to promote th...,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot..."
323,16.5.1,Proportion of individuals using the Internet.,"[individual, using, internet]",proportion of individuals using the internet,[]
324,17.1.1,Proportion of sustainable development indicato...,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc..."
325,17.2.1,Dollar value of all resources made available t...,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va..."


In [38]:
# Apply the function to each row of the DataFrame
sdg_sem = sdg_sem.apply(add_elements, axis=1)
sdg_sem

,Id,Description,new_description,new_description_str,keywords
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge..."
...,...,...,...,...,...
322,16.4.1,Total amount of approved funding to promote th...,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot..."
323,16.5.1,Proportion of individuals using the Internet.,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]"
324,17.1.1,Proportion of sustainable development indicato...,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc..."
325,17.2.1,Dollar value of all resources made available t...,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va..."


In [39]:
print(sdg_sem['Description'][11])

Ensure sustainable consumption and production patterns


In [57]:
sdg_sem.to_csv(absolute_path_sdg, sep=";", index=False)